In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!git clone https://gitlab.com/mjslee0921/proteinsgm.git

# Unzip the contents of the archive directly into /usr/local/envs/
!tar -xzvf /content/drive/MyDrive/Generative_Models/envs/proteinsgm.tar.gz -C /usr/local/envs/ >/dev/null 2>&1

In [ ]:
"""
%cd ./proteinsgm
!conda env create -f env.yaml

In [ ]:
"""
%%bash
source activate proteinsgm
conda uninstall pytorch -y
conda uninstall cudatoolkit -y
conda install cudatoolkit==11.8 -y
conda install pytorch torchvision torchaudio pytorch-cuda=11.8 cuda=11.8 -c pytorch -c nvidia -y
conda clean --all -y

In [ ]:
"""
!tar -czvf /content/proteinsgm.tar.gz -C /usr/local/envs/ .
!mv /content/proteinsgm.tar.gz /content/drive/MyDrive/Generative_Models/envs/proteinsgm.tar.gz


In [ ]:
%cd ./proteinsgm

/content/proteinsgm


In [ ]:
drive.mount('/content/drive')
import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/unconditional_generation/proteinsgm_unconditional/generation_metadata_proteinsgm.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created generation metadata dataframe")


len_dist_filepath = "/content/drive/MyDrive/Generative_Models/unconditional_generation/proteinsgm_unconditional/uniref50_length_dist_proteinsgm.json"

if os.path.exists(len_dist_filepath):
  with open(len_dist_filepath, "r") as f:
    uniprot_length_dist =  json.load(f)
  print("Loaded length distribution from drive")
else:

  #https://www.uniprot.org/uniprotkb/statistics#sequence-size
  bins = np.array([13,51,101,151,201,251,301,351,401,451,501,551,601,651,701,751,801,851,901,951,1001,1101,1201,1301,1401,1501,1601,1701,1801,1901,2001,2101,2201,2301,2401,2501,34350])
  swissprot_reviewed = np.array([0,9968,43534,59796,59574,58452,52413,52846,45901,37706,30572,22287,15830,13156,9403,7870,5700,4889,5301,4109,3007,4124,2897,2207,2070,1675,834,642,587,503,395,272,386,340,234,195,1462])
  TrEMBL_unreviewed = np.array([0,2668805,19825275,24705701,23838128,23462438,23225451,21389271,16814580,14287105,11501843,8283150,6266068,4715059,3755005,3186452,2687314,2166878,1843669,1457871,1153537,1975953,1398765,961048,664766,517536,390552,300984,236895,210921,180246,138808,122833,102865,82441,71548,527646])

  ecdf = np.cumsum(swissprot_reviewed) / np.sum(swissprot_reviewed)
  #shortest protein in uniprot is 14 res, longest is 34350 res.
  x = np.arange(14, 34350+1)
  ecdf = np.interp(x, bins, ecdf)

  # Sample from the empirical CDF
  num_samples = 11000
  random_values = np.random.rand(num_samples)
  sampled_lengths = np.round(np.interp(random_values, ecdf, x)).astype(int)
  #ten thousand sequences up to 1000 res in length
  sampled_lengths = sampled_lengths[sampled_lengths <= 1000][0:10000]

  # Plot the histogram of sampled values
  hist_values, bin_edges, patches = plt.hist(sampled_lengths, bins=x[0:1001-13], alpha=0.7, label='Sampled Values')
  plt.xlabel('X-axis label')
  plt.ylabel('Frequency')
  plt.legend()
  plt.show()

  uniprot_length_dist = list(zip([int(edge) for edge in bin_edges],[int(value) for value in hist_values]))
  with open(len_dist_filepath, "w") as f:
      json.dump(uniprot_length_dist, f)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Existing generation metadata read in.
Loaded length distribution from drive


In [ ]:
def make_generation_command(length,batch_size):
  return f"""
source activate proteinsgm
python -c '

import torch
torch.cuda.set_device(0)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

import numpy as np
from pathlib import Path
from score_sde_pytorch.utils import get_model, restore_checkpoint, recursive_to
from score_sde_pytorch.models.ema import ExponentialMovingAverage
import score_sde_pytorch.sde_lib as sde_lib
import score_sde_pytorch.sampling as sampling
import score_sde_pytorch.losses as losses
import pickle as pkl
import argparse
import yaml
from easydict import EasyDict
from tqdm.auto import tqdm
from utils import get_conditions_random, get_mask_all_lengths, get_conditions_from_pdb
import time

torch.cuda.set_device(0)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

with open("./configs/cond_length.yml", "r") as f:
        config = EasyDict(yaml.safe_load(f))
config.device = "cuda"

workdir = Path("sampling", "coords_6d", Path("./configs/cond_length.yml").stem, Path("./checkpoints/cond_length.pth").stem, "len_"+ str({length}))


score_model = get_model(config)
ema = ExponentialMovingAverage(score_model.parameters(), decay=config.model.ema_rate)
optimizer = losses.get_optimizer(config, score_model.parameters())
state = dict(optimizer=optimizer, model=score_model, ema=ema, step=0)
state = restore_checkpoint("./checkpoints/cond_length.pth", state, "cuda")
state["ema"].store(state["model"].parameters())
state["ema"].copy_to(state["model"].parameters())
print("Start time " + str(time.time()))

if config.training.sde == "vesde":
    sde = sde_lib.VESDE(sigma_min=config.model.sigma_min, sigma_max=config.model.sigma_max,
                        N=config.model.num_scales)
    sampling_eps = 1e-5
elif config.training.sde == "vpsde":
    sde = sde_lib.VPSDE(beta_min=config.model.beta_min, beta_max=config.model.beta_max, N=config.model.num_scales)
    sampling_eps = 1e-3

sampling_shape = ({batch_size}, config.data.num_channels,
                      config.data.max_res_num, config.data.max_res_num)

sampling_fn = sampling.get_sampling_fn(config, sde, sampling_shape, sampling_eps)

generated_samples = []

for _ in tqdm(range(1)):
  mask = get_mask_all_lengths(config,batch_size={batch_size})[{length}-40]
  condition = {{"length": mask.to(config.device)}}
  sample, n = sampling_fn(state["model"], condition)
  generated_samples.append(sample.cpu())

workdir.mkdir(parents=True, exist_ok=True)
with open(workdir.joinpath("len_"+ str({length})+".pkl"), "wb") as f:
  pkl.dump(generated_samples, f)
print("End time " + str(time.time()))
'
"""

In [ ]:

import time
for length, batch_size in uniprot_length_dist:
  if batch_size == 0: continue
  if length <40: continue
  if length > 128: continue
  if all_metadata_df.loc[(all_metadata_df.conditions == "length = " + str(length)) & (all_metadata_df.gpu == "T4 GPU"),:].shape[0] >= 1: continue
  #if all_metadata_df.loc[all_metadata_df.conditions == "length = " + str(length),:].shape[0] >= batch_size: continue

  cleanup_command = f"""
  for file in /content/proteinsgm/sampling/coords_6d/cond_length/cond_length/len_{length}/*
  do
    bn=$(basename "$file")
    mv $file /content/drive/MyDrive/Generative_Models/unconditional_generation/proteinsgm_unconditional/$bn
  done
  rm -rf /content/proteinsgm/sampling
  """
  generation_command = make_generation_command(length, batch_size)
  meta_data = {}
  meta_data['batch_id'] = str(uuid.uuid4())
  meta_data['batch_size'] = str(batch_size)
  meta_data['Timestamp'] = str(datetime.now())
  meta_data['model'] = 'ProteinSGM'
  meta_data['task'] = 'backbone generation (6D)'
  meta_data['conditions'] = 'length = ' + str(length)
  meta_data['gpu'] = 'T4 GPU'
  start_time = time.time()
  !{generation_command}
  end_time = time.time()
  total_job_time = end_time - start_time
  meta_data['wall_time_batch'] = str(total_job_time) + " Seconds"
  meta_data['wall_time_task'] = str(total_job_time/batch_size) + " Seconds (inferred)"
  for filename in os.listdir("/content/proteinsgm/sampling/coords_6d/cond_length/cond_length/len_"+str(length)):
        meta_data['output_file_name'] = filename
        metadata_entry = pd.Series(meta_data)
        all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(metadata_entry).T], ignore_index=True)

  all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Metadata saved. Cleaning up....")
  !{cleanup_command}
  torch.cuda.empty_cache()


In [ ]:
#The rosetta minimasation step runs on the cpu so we'll do it as a seperate step (Disconnect and Reconnect runtime). It also doesn't require the huge conda environment we ended up with to get the 6D sampling to work.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!git clone https://gitlab.com/mjslee0921/proteinsgm.git


Mounted at /content/drive
--2024-05-29 23:29:28--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143808873 (137M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 137.15M   257MB/s    in 0.5s    

2024-05-29 23:29:29 (257 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [143808873/143808873]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please

In [ ]:
%cd ./proteinsgm
!conda env create -f env.yaml


/content/proteinsgm
Channels:
 - defaults
 - pytorch
 - conda-forge
 - https://conda.rosettacommons.org
Platform: linux-64
Solving environment: / - \ | / done

pyrosetta-2024.19+re | 1.50 GB   | :   0% 0/1 [00:00<?, ?it/s]
cudatoolkit-11.3.1   | 549.3 MB  | :   0% 0/1 [00:00<?, ?it/s]

mysql-5.7.24         | 60.0 MB   | :   0% 0/1 [00:00<?, ?it/s]


qt-main-5.15.2       | 53.7 MB   | :   0% 0/1 [00:00<?, ?it/s]



pytorch-1.12.1       | 49.2 MB   | :   0% 0/1 [00:00<?, ?it/s]




libllvm14-14.0.6     | 33.4 MB   | :   0% 0/1 [00:00<?, ?it/s]





biotite-0.35.0       | 33.2 MB   | :   0% 0/1 [00:00<?, ?it/s]






python-3.8.19        | 23.8 MB   | :   0% 0/1 [00:00<?, ?it/s]







scipy-1.10.1         | 22.4 MB   | :   0% 0/1 [00:00<?, ?it/s]








pandas-2.0.3         | 12.4 MB   | :   0% 0/1 [00:00<?, ?it/s]









libclang13-14.0.6    | 9.8 MB    | :   0% 0/1 [00:00<?, ?it/s]










bokeh-2.4.3          | 7.7 MB    | :   0% 0/1 [00:00<?, ?it/s]











numpy-ba

In [ ]:
drive.mount('/content/drive')
import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/unconditional_generation/proteinsgm_unconditional/generation_metadata_proteinsgm.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created generation metadata dataframe")


len_dist_filepath = "/content/drive/MyDrive/Generative_Models/unconditional_generation/proteinsgm_unconditional/uniref50_length_dist_proteinsgm.json"

if os.path.exists(len_dist_filepath):
  with open(len_dist_filepath, "r") as f:
    uniprot_length_dist =  json.load(f)
  print("Loaded length distribution from drive")
else:

  #https://www.uniprot.org/uniprotkb/statistics#sequence-size
  bins = np.array([13,51,101,151,201,251,301,351,401,451,501,551,601,651,701,751,801,851,901,951,1001,1101,1201,1301,1401,1501,1601,1701,1801,1901,2001,2101,2201,2301,2401,2501,34350])
  swissprot_reviewed = np.array([0,9968,43534,59796,59574,58452,52413,52846,45901,37706,30572,22287,15830,13156,9403,7870,5700,4889,5301,4109,3007,4124,2897,2207,2070,1675,834,642,587,503,395,272,386,340,234,195,1462])
  TrEMBL_unreviewed = np.array([0,2668805,19825275,24705701,23838128,23462438,23225451,21389271,16814580,14287105,11501843,8283150,6266068,4715059,3755005,3186452,2687314,2166878,1843669,1457871,1153537,1975953,1398765,961048,664766,517536,390552,300984,236895,210921,180246,138808,122833,102865,82441,71548,527646])

  ecdf = np.cumsum(swissprot_reviewed) / np.sum(swissprot_reviewed)
  #shortest protein in uniprot is 14 res, longest is 34350 res.
  x = np.arange(14, 34350+1)
  ecdf = np.interp(x, bins, ecdf)

  # Sample from the empirical CDF
  num_samples = 11000
  random_values = np.random.rand(num_samples)
  sampled_lengths = np.round(np.interp(random_values, ecdf, x)).astype(int)
  #ten thousand sequences up to 1000 res in length
  sampled_lengths = sampled_lengths[sampled_lengths <= 1000][0:10000]

  # Plot the histogram of sampled values
  hist_values, bin_edges, patches = plt.hist(sampled_lengths, bins=x[0:1001-13], alpha=0.7, label='Sampled Values')
  plt.xlabel('X-axis label')
  plt.ylabel('Frequency')
  plt.legend()
  plt.show()

  uniprot_length_dist = list(zip([int(edge) for edge in bin_edges],[int(value) for value in hist_values]))
  with open(len_dist_filepath, "w") as f:
      json.dump(uniprot_length_dist, f)


In [ ]:
def make_minimisation_command(data,length,index):
  return f"""
source activate proteinsgm
python -c '

import math
import numpy as np
from pathlib import Path
import pickle as pkl
from pyrosetta import *
import rosetta_min.run as rosetta
import argparse
import time

#Arguments
data = {data}
tag="len_"+ str({length})
index={index}
mask_info="1:5,10:15"
n_iter=10
dist_std=2
angle_std=20
fastdesign=False
fastrelax=False

outPath = Path("sampling", "rosetta", tag,str(Path(data).parent.stem)+"_index_"+str(index))
with open(data, "rb") as f:
  samples = pkl.load(f)[0] #for some reason had to alter this to take one-and-only object from pickle
sample = samples[index]
msk = np.round(sample[-1])
L = math.sqrt(len(msk[msk == 1]))
if not (L).is_integer():
  raise ValueError("Terminated due to improper masking channel...")
else:
  L = int(L)

# Initialize sequence of polyalanines and gather constraints
seq = "A" * L
pose = None

npz = {{}}
for idx, name in enumerate(["dist", "omega", "theta", "phi"]):
  npz[name] = np.clip(sample[idx][msk == 1].reshape(L, L), -1, 1)

npz["dist_abs"] = (npz["dist"] + 1) * 10
npz["omega_abs"] = npz["omega"] * math.pi
npz["theta_abs"] = npz["theta"] * math.pi
npz["phi_abs"] = (npz["phi"] + 1) * math.pi / 2

rosetta.init_pyrosetta()

for n in range(n_iter):
  outPath_run = outPath.joinpath("round_"+str(n + 1))
  if outPath_run.joinpath("final_structure.pdb").is_file():
      continue

  _ = rosetta.run_minimization(
      npz,
      seq,
      pose=pose,
      scriptdir=Path("rosetta_min"),
      outPath=outPath_run,
      angle_std=angle_std,  # Angular harmonic std
      dist_std=dist_std,  # Distance harmonic std
      use_fastdesign=fastdesign,
      use_fastrelax=fastrelax,
  )

if fastdesign:
  score_fn = create_score_function("ref2015").score
  filename = "final_structure.pdb" if fastrelax else "structure_after_design.pdb"
else:
  score_fn = ScoreFunction()
  score_fn.add_weights_from_file(str(Path("rosetta_min").joinpath("data/scorefxn_cart.wts")))
  filename = "structure_before_design.pdb"

e_min = 9999
best_run = 0
for i in range(n_iter):
  pose = pose_from_pdb(str(outPath.joinpath("round_"+str(i+1), filename)))
  e = score_fn(pose)
  if e < e_min:
    best_run = i
    e_min = e

outPath.joinpath(f"best_run").symlink_to(outPath.joinpath("round_"+str(best_run + 1)).resolve(),target_is_directory=True)
with open(outPath.joinpath("sample.pkl"), "wb") as f:
  pkl.dump(sample, f)
'
"""

In [ ]:
import time
from collections import defaultdict
to_minimise = all_metadata_df.loc[all_metadata_df.task == "backbone generation (6D)",:]
to_minimise['length'] = to_minimise['conditions'].str.extract(r'(\d+)').astype(int)
to_minimise = to_minimise.loc[(to_minimise.length<=128) & (to_minimise.length>=40),:].sort_values(by='Timestamp', ascending=False).drop_duplicates(subset='output_file_name', keep='first').reset_index(drop=True)

already_done = all_metadata_df.loc[all_metadata_df.task == "backbone generation (Rosetta)",:]
already_done['length'] = already_done['conditions'].str.extract(r'(\d+)').astype(int)
already_done = defaultdict(int, already_done.groupby('length').size().to_dict())

#to_minimise[~to_minimise['conditions'].isin(all_metadata_df[all_metadata_df.task == "backbone generation (Rosetta)"].conditions.unique())]
for index,row in to_minimise.iterrows():
  length = row['length']
  batch_size = int(row['batch_size'])
  pickle = "/content/drive/MyDrive/Generative_Models/unconditional_generation/proteinsgm_unconditional/" + row['output_file_name']
  if batch_size == 0: continue
  print(pickle)
  for i in range(already_done[length],batch_size):
    print("generating " + "len"+str(length)+"_"+str(i)+".pdb .....")
    cleanup_command = f"""
    for file in /content/proteinsgm/sampling/rosetta/len_{length}/proteinsgm_unconditional_index_{i}/round_10/*
    do
      bn=$(basename "$file")
      mv $file /content/drive/MyDrive/Generative_Models/unconditional_generation/proteinsgm_unconditional/$bn
    done
    rm -rf /content/proteinsgm/sampling
    """
    minimisation_command = make_minimisation_command('"'+pickle+'"', length, i)
    meta_data = {}
    meta_data['batch_id'] = None
    meta_data['batch_size'] = None
    meta_data['Timestamp'] = str(datetime.now())
    meta_data['model'] = 'ProteinSGM'
    meta_data['task'] = 'backbone generation (Rosetta)'
    meta_data['conditions'] = 'length = ' + str(length)
    meta_data['gpu'] = 'CPU'
    start_time = time.time()
    !{minimisation_command}
    end_time = time.time()
    total_job_time = end_time - start_time
    meta_data['wall_time_task'] = str(total_job_time) + " Seconds"
    for filename in os.listdir("/content/proteinsgm/sampling/rosetta/len_"+str(length)+"/proteinsgm_unconditional_index_" + str(i) + "/round_10/"):
      meta_data['entity_id'] = str(uuid.uuid4())
      new_file_name = "ProteinSGM_len" + str(length) + "_" + meta_data['entity_id'] + ".pdb
      shutil.move("/content/proteinsgm/sampling/rosetta/len_"+str(length)+"/proteinsgm_unconditional_index_" + str(i) + "/round_10/" + filename,"/content/proteinsgm/sampling/rosetta/len_"+str(length)+"/proteinsgm_unconditional_index_" + str(i) + "/round_10/" + new_file_name)
      meta_data['output_file_name'] = new_file_name
      metadata_entry = pd.Series(meta_data)
      all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(metadata_entry).T], ignore_index=True)
    all_metadata_df.to_csv(meta_data_filepath, index=False)
    print("Metadata saved. Cleaning up....")
    !{cleanup_command}
